In [14]:
# 1. Librerías necesarias
import pandas as pd
from pathlib import Path

# Ruta base a tus datos raw
RAW_DATA_DIR = Path("/content")

# Posadas
path_posadas = RAW_DATA_DIR / "fake_news_corpus_posadas_full.csv"
# Omdena
path_omdena = RAW_DATA_DIR / "fake_news_latam_omdena_combined.csv"


In [17]:
# --- 3. Cargar POSADAS ---
df_posadas = pd.read_csv(path_posadas)

# Nos quedamos con las columnas útiles ya en minúsculas
df_posadas = df_posadas[['category', 'text', 'headline', 'source', 'split']].copy()

# Renombrar para unificar con Omdena
df_posadas = df_posadas.rename(columns={
    'category': 'label',
    'text': 'content',
    'headline': 'title',
    'source': 'source'
})

# Añadir columna de origen
df_posadas['dataset'] = 'posadas'

# Estandarizar etiquetas
df_posadas['label'] = df_posadas['label'].map({'Fake': 'Falsa', 'True': 'Verdadera'})


In [18]:
# --- 4. Cargar OMDENA ---
df_omdena = pd.read_csv(path_omdena)

df_omdena = df_omdena[['Corrected_label', 'Title', 'Text', 'Source']]
df_omdena = df_omdena.rename(columns={
    'Corrected_label': 'label',
    'Title': 'title',
    'Text': 'content',
    'Source': 'source'
})
df_omdena['dataset'] = 'omdena'
df_omdena['label'] = df_omdena['label'].map({'Fake': 'Falsa', 'True': 'Verdadera'})

In [19]:
# --- 5. Unificación y limpieza ---
df_combined = pd.concat([df_posadas, df_omdena], ignore_index=True)
df_combined = df_combined.dropna(subset=['content', 'label'])
df_combined = df_combined[df_combined['label'].isin(['Falsa', 'Verdadera'])]
df_combined = df_combined.reset_index(drop=True)



In [20]:
print(f"Posadas: {len(df_posadas)} noticias")
print(f"Omdena: {len(df_omdena)} noticias")
print(f"Combinado: {len(df_combined)} noticias")

Posadas: 1543 noticias
Omdena: 3092 noticias
Combinado: 4349 noticias


In [21]:
# Noticias con contenido o etiqueta faltante en cada dataset
print("Filas con content o label vacíos:")
print("Posadas:", df_posadas[['content', 'label']].isna().any(axis=1).sum())
print("Omdena:", df_omdena[['content', 'label']].isna().any(axis=1).sum())


Filas con content o label vacíos:
Posadas: 286
Omdena: 0


In [22]:
# Proporciones globales
print("Proporciones globales combinadas:")
display(df_combined['label'].value_counts(normalize=True))

# Proporciones individuales
print("\nPOSADAS:")
display(df_posadas['label'].value_counts(normalize=True))

print("\nOMDENA:")
display(df_omdena['label'].value_counts(normalize=True))


Proporciones globales combinadas:


,proportion
label,
Verdadera,0.654633
Falsa,0.345367



POSADAS:


,proportion
label,
Verdadera,0.618138
Falsa,0.381862



OMDENA:


,proportion
label,
Verdadera,0.66947
Falsa,0.33053


In [23]:
# --- 6. Guardar CSV procesado ---
output_path = Path("/content/fake_news_combined_clean.csv")
output_path.parent.mkdir(parents=True, exist_ok=True)
df_combined.to_csv(output_path, index=False)
print(f"✅ Dataset limpio guardado en: {output_path}")

✅ Dataset limpio guardado en: /content/fake_news_combined_clean.csv
